In [ ]:
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

In [ ]:
import keras

In [ ]:
model = keras.models.load_model("/content/drive/Shareddrives/Project_Engineering_Success/Project_Engineering_Success.h5")

In [ ]:
#@markdown We implemented some functions to visualize the hand landmark detection results. <br/> Run the following cell to activate the functions.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

In [ ]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


# STEP 2: Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)



In [ ]:

def convert2numpy(detection_result):
    hand_landmarks_list = detection_result.hand_landmarks
    # Check if the list is not empty
    if not hand_landmarks_list:
        print("No landmarks detected!")
        # Return a padded array of zeros (21 landmarks x 3 coordinates)
        return np.zeros((21, 3), dtype=np.float32)

    # Assuming at least one hand is detected, we extract the first hand's landmarks
    landmark_list = [
        (landmark.x, landmark.y, landmark.z)
        for landmark in hand_landmarks_list[0]  # Iterate through the first hand's landmarks
    ]
    # Convert to NumPy array
    landmark_array = np.array(landmark_list, dtype=np.float32)
    return landmark_array



In [ ]:
import numpy as np
import tensorflow as tf

# Define the label map (reverse mapping)
label_map = {0: 'Right', 1: 'Down', 2: 'Fist', 3: 'Front',
             4: 'Left', 5: 'None', 6: 'Up', 7: 'Back'}

# Function to decode one-hot encoded labels
def decode_label(one_hot_vector):
    return label_map[np.argmax(one_hot_vector)]  # Get the class index and map it

# Perform inference on an image
def predict_image(image_path):
    image = mp.Image.create_from_file(image_path)
    detection_result = detector.detect(image)
    data=convert2numpy(detection_result)
    batched_data = np.expand_dims(data, axis=0)

    # Get prediction
    prediction = model.predict(batched_data)
    predicted_label = decode_label(prediction[0])  # Decode prediction

    return predicted_label

# Test the function
image_path = "/content/drive/Shareddrives/Project_Engineering_Success/rename_resizedataset3/Back/image_0002.jpg"
predicted_gesture = predict_image(image_path)

print(f"Predicted Gesture: {predicted_gesture}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
Predicted Gesture: Back


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import time

def take_photo_at_fps(filename='photo.jpg', fps=10, quality=0.8):
    js = Javascript('''
    async function takePhoto(quality, fps) {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = 'Capture';
        div.appendChild(capture);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        let captureInterval = setInterval(async function() {
            // Capture a frame every fps seconds
            capture.click();
        }, 1000 / fps);  // Interval in milliseconds (1 / fps)

        // Wait for Capture to be clicked
        await new Promise((resolve) => capture.onclick = resolve);

        clearInterval(captureInterval);  // Stop the interval after capturing

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
    }
    ''')

    display(js)
    # Capture the photo after a specified time interval (fps)
    data = eval_js(f'takePhoto({quality}, {fps})')
    binary = b64decode(data.split(',')[1])

    # Save the captured image to the specified file
    with open(filename, 'wb') as f:
        f.write(binary)

    return filename


In [ ]:
from PIL import Image

def detector():
  filename = take_photo_at_fps()
  image = Image.open(filename)
  predicted_gesture = predict_image(filename)
  return predicted_gesture

def 

while True:
  detector()

<IPython.core.display.Javascript object>

No landmarks detected!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Predicted Gesture: Fist


<IPython.core.display.Javascript object>

No landmarks detected!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predicted Gesture: Fist


<IPython.core.display.Javascript object>

...

No landmarks detected!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predicted Gesture: Fist


<IPython.core.display.Javascript object>

No landmarks detected!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Predicted Gesture: Fist


<IPython.core.display.Javascript object>

KeyboardInterrupt: 